In [1]:
#export
"""This module for tools that will likely start the processing stream."""
from typing import Iterator, Union, Any, List
import k1lib, urllib, subprocess, warnings, os, k1lib, threading, time, warnings, math, io, dill, urllib, validators
from collections import deque
from k1lib.cli import BaseCli; import k1lib.cli as cli
from k1lib.cli.typehint import *
from contextlib import contextmanager
requests = k1lib.dep("requests")
try: import minio; hasMinio = True
except: hasMinio = False
__all__ = ["cat", "splitSeek", "refineSeek", "wget", "ls", "cmd", "walk", "requireCli", "urlPath", "kzip", "kunzip", "unzip"]

In [2]:
#export
settings = k1lib.settings.cli
class NoPartialContent(Exception): pass
def getChunk(url:str, sB:int, eB:int, timeout:float, retries:int) -> bytes: # start inclusive, end exclusive
    for i in range(retries):
        try: res = requests.get(url, headers={"Range": f"bytes={sB}-{eB-1}"}, timeout=timeout)
        except Exception as e:
            if i >= retries-1: raise Exception(f"Can't get file chunk")
            continue
        if res.status_code != 206: raise NoPartialContent(f"Server doesn't allow partial downloads at this particular url. Status code: {res.status_code}")
        return res.content
def getChunks(url:str, sB:int, eB:int, chunkSize=None, chunkTimeout:float=10, chunkRetries:int=10) -> List[bytes]:
    """Grabs bytes from sB to eB in chunks"""
    chunkSize = chunkSize or settings.cli.cat.chunkSize
    return range(sB, eB+1) | cli.batched(chunkSize, True) | cli.apply(lambda r: getChunk(url, r.start, r.stop-1, chunkTimeout, chunkRetries))

In [3]:
#export
catSettings = k1lib.Settings().add("chunkSize", 100000, "file reading chunk size for binary+chunk mode. Decrease it to avoid wasting memory and increase it to avoid disk latency")
catSettings.add("every", k1lib.Settings().add("text", 1000, "for text mode, will print every n lines").add("binary", 10, "for binary mode, will print every n 100000-byte blocks"), "profiler print frequency")
settings.add("cat", catSettings, "inp.cat() settings")

rfS = k1lib.Settings()
settings.add("RemoteFile", rfS, "inp.RemoteFile() settings, used in cat(), splitSeek() and the like")
rfS.add("memoryLimit", 100_000_000, "if the internal cache exceeds this limit (in bytes), and randomAccess is False, then old downloaded chunks will be deleted")
rfS.add("timeout", 10, "seconds before terminating the remote request and retrying")
rfS.add("retries", 10, "how many times to retry sending the request before giving up")

Settings:                   
- memoryLimit = 100000000   ​if the internal cache exceeds this limit (in bytes), and randomAccess is False, then old downloaded chunks will be deleted   
- timeout     = 10          ​seconds before terminating the remote request and retrying                                                                   
- retries     = 10          ​how many times to retry sending the request before giving up                                                                 
                            

In [4]:
#export
def noPartial(url, *args):
    try: return len(getChunk(url, 0, 10, *args)) != 10
    except NoPartialContent: return True
class RemoteFile:
    def __init__(self, url, randomAccess=True, blockSize=None, noPartialConfirm=False, timeout:float=None, retries:int=None):
        """
:param url: url of the remote file
:param randomAccess: is random accessing parts of the file expected? If
    True, then keeps all of the reads in ram internally, else free them
    as soon as possible
:param blockSize: all reads will fetch roughly this amount of bytes"""
        self.url = url; self.randomAccess = randomAccess; self.blockSize = blockSize or settings.cat.chunkSize
        self.noPartialConfirm = noPartialConfirm; self.size = None; self.domain = k1lib.Domain()
        self.seekPos = 0; self.reads = deque() # List[sB, eB, content]
        self._confirmMsgShown = False; self.timeout = timeout or rfS.timeout; self.retries = retries or rfS.retries
        self._totalReadSize = 0; self.noPartial = noPartial(url, self.timeout, self.retries)
    def _fetch(self, sB:int, eB:int): # fetches from start to end byte and dumps to internal memory. Inclusive start and end byte
        if not self.noPartial:
            eB = max(eB, min(sB+self.blockSize, len(self)))
            chunk = getChunk(self.url, sB, eB, self.timeout, self.retries)
        else:
            if self.noPartialConfirm and not self._confirmMsgShown:
                ans = input(f"""Remote file '{self.url}' don't support partial downloads.
Therefore the entire file will be loaded into RAM, which
could be undesireable. Do you want to continue? Y/n: """)
                self._confirmMsgShown = True
                if ans.lower()[0] != "y": self.reads.append([0, 0, b""]); return
            sB = 0; chunk = requests.get(self.url).content; eB = len(chunk)
        self.reads.append([sB, eB, chunk])
        self._totalReadSize += len(chunk); self.domain = self.domain + k1lib.Domain([sB, eB])
        if not self.randomAccess and self._totalReadSize > rfS.memoryLimit: # deletes old reads
            sB, eB, chunk = self.reads.popleft()
            self._totalReadSize -= len(chunk)
    def _ensureRange(self, sB, eB): # makes sure that all ranges between sB and eB are available
        missingDomain = k1lib.Domain([max(sB-30, 0), min(eB+30, len(self))]) & -self.domain
        for sB, eB in missingDomain.ranges: self._fetch(sB, eB)
    def _readChunks(self, sB, eB): # read from sB to eB, but in chunks, to be optimized. inclusive sB, exclusive eB
        sB = max(min(sB, len(self)), 0); eB = max(min(eB, len(self)), 0); self._ensureRange(sB, eB)
        return self.reads | cli.filt(~cli.aS(lambda s,e,chunk: e>=sB and s<=eB)) | cli.sort() | ~cli.apply(lambda s,e,chunk: chunk[max(sB-s, 0):len(chunk)+min(eB-e, 0)]) | cli.filt(len)
    def seek(self, cookie, whence=0):
        if whence == 0: self.seekPos = cookie
        elif whence == 1: self.seekPos += cookie
        elif whence == 2: self.seekPos = len(self) + cookie
        else: raise Exception("Invalid whence")
        return self.seekPos
    def read(self, size, join=True):
        chunks = self._readChunks(self.seekPos, self.seekPos + size)
        self.seekPos += size; return b"".join(chunks) if join else chunks
    def readline(self, newLine=True):
        ans = []; seekPos = self.seekPos
        try:
            while self.seekPos < len(self):
                for chunk in self.read(self.blockSize, False):
                    if len(chunk) == 0: raise SyntaxError()
                    ans.append(chunk)
                    if b"\n" in chunk: raise SyntaxError()
        except SyntaxError: pass
        ans = b"".join(ans)
        try: n = ans.index(b"\n")
        except ValueError: n = len(ans) # only happens at end of file
        self.seekPos = seekPos + n+1; return (ans[:n+1] if newLine else ans[:n]).decode()
    def readlines(self, newLine=True):
        while True:
            yield self.readline(newLine)
            if self.seekPos >= len(self): break
    def tell(self): return self.seekPos
    def _getSize(self):
        if self.noPartial: self._fetch(0, 10); return self.reads[0][1]
        for i in range(self.retries):
            try: return requests.head(self.url, timeout=self.timeout).headers.items() | cli.apply(cli.op().lower(), 0) | cli.toDict() | cli.op()["content-length"].ab_int()
            except Exception as e:
                if i >= self.retries: raise Exception(f"Can't get size of remote file: {e}")
    def __len__(self):
        if self.size is None: self.size = self._getSize()
        return self.size
    def __repr__(self): return f"<RemoteFile url={self.url} size={k1lib.fmt.size(len(self))}>"

In [5]:
url = "http://vim.kelvinho.org/.vimrc"; rf = RemoteFile(url, True, 30, True)
a = rf.readlines(False) | cli.deref(); b = requests.get(url).text.split("\n")[:-1]; assert a == b

In [6]:
#export
import zipfile, inspect
class ZipWrapper:
    def __init__(self, a, zfn): self.a = a; self.zfn = zfn
    def __repr__(self):
        a = self.a; s = f" ({round(a.compress_size/a.file_size*100)}%)" if a.file_size > 0 else ""
        return f"<Zip subfile name='{a.filename}' {k1lib.fmt.size(a.file_size)} -> {k1lib.fmt.size(a.compress_size)}{s}>"
    @property
    def size(self): return a.file_size
    @property
    def compressedSize(self): return a.compress_size
    def _catHandle(self):
        with zipfile.ZipFile(self.zfn) as zipf:
            with zipf.open(self.a.filename) as subfile:
                yield subfile
@contextmanager
def openFile(fn, text, noPartialConfirm=False): # can be actual file or url
    if not isinstance(fn, str):
        if hasattr(fn, "_catHandle"): yield from fn._catHandle(); return # custom datatype case
        else: yield fn; return # file handle case, just return itself
    if os.path.exists(fn):
        if text:
            with open(fn, "r", settings.cat.chunkSize) as f: yield f
        else:
            with open(fn, "rb", settings.cat.chunkSize) as f: yield f
    elif validators.url(fn) is True:
        yield RemoteFile(fn, False, noPartialConfirm=noPartialConfirm)
    else: raise FileNotFoundError(f"The file {fn} doesn't seem to exist and/or it's not a valid url")
def _catGenText(fn, sB, eB): # fn for "file name"
    try:
        if sB == 0 and eB == -1: # fast path without bounds (90-160 MB/s expected)
            with openFile(fn, True) as f:
                line = f.readline()
                if isinstance(line, str): # why put the if outside the loop? Speed reasons. Also, isn't .readline() supposed to return a string? Well, cause we're supporting random file handles from custom datatypes, some asshole file handles might return bytes instead
                    while True:
                        if line == "": return
                        yield line[:-1] if line[-1] == "\n" else line
                        line = f.readline()
                else:
                    while True:
                        line = line.decode()
                        if line == "": return
                        yield line[:-1] if line[-1] == "\n" else line
                        line = f.readline()
        else: # slow path with bounds (15 MB/s expected). Update: much faster now, expect only 40% slower than the path above
            sB = wrap(fn, sB); eB = wrap(fn, eB)
            with openFile(fn, True) as f:
                f.seek(sB); b = sB # current byte
                line = f.readline()
                if isinstance(line, str):
                    while True:
                        b += len(line)
                        if len(line) == 0: return
                        if b > eB: yield line[:len(line)-(b-eB)]; return
                        yield line[:-1] if line[-1] == "\n" else line
                        line = f.readline()
                else:
                    while True:
                        line = line.decode()
                        b += len(line)
                        if len(line) == 0: return
                        if b > eB: yield line[:len(line)-(b-eB)]; return
                        yield line[:-1] if line[-1] == "\n" else line
                        line = f.readline()
    except FileNotFoundError: pass
def _catGenBin(fn, sB, eB):
    chunkSize = settings.cat.chunkSize; sB = wrap(fn, sB); eB = wrap(fn, eB); nB = eB - sB # number of bytes to read total
    with openFile(fn, False) as f:
        f.seek(sB); nChunks = math.ceil(nB / chunkSize); lastChunkSize = nB - chunkSize*(nChunks-1)
        # had to do this because RemoteFile is actually not thread-safe
        # applyF = (lambda f_: cli.apply(f_)) if isinstance(f, RemoteFile) else (lambda f_: cli.applyTh(f_, prefetch=10))
        applyF = (lambda f_: cli.apply(f_)) # actually, normal reads are not thread-safe either. Stupid me
        yield from range(nChunks) | applyF(lambda i: f.read(chunkSize) if i < nChunks-1 else f.read(chunkSize)[:lastChunkSize])
def fileLength(fn):
    with openFile(fn, False) as f: return f.seek(0, os.SEEK_END)
def wrap(fn, b): return b if b >= 0 else b + fileLength(fn) + 1
class _cat(BaseCli):
    def __init__(self, text, chunks, sB, eB):
        super().__init__(capture=True)
        self.text = text; self.chunks = chunks; self.sB = sB; self.eB = eB
    def _typehint(self, ignored=None):
        if self.text: return tIter(str) if self.chunks else tList(str)
        else: return tIter(bytes) if self.chunks else bytes
    def __ror__(self, fn:Union[str, "fileHandle"]) -> Union[Iterator[str], bytes]:
        ser = self.capturedSerial; text = self.text; chunks = self.chunks; sB = self.sB; eB = self.eB
        if not isinstance(fn, str) and hasattr(fn, "_cat"): # custom datatype, _cat method defined, so it will take control of things
            kwargs = {"text": text, "chunks": chunks, "sB": sB, "eB": eB}
            kwdiff = [a for a, b in [["text",text!=True], ["chunks",chunks!=True if text else chunks!=False], ["sB",sB!=0], ["eB",eB!=-1]] if b]
            args = inspect.getfullargspec(fn._cat).args[1:]; n = len(args)
            s = set(["ser", "kwargs"]); weirdArgs = [a for a in args if a not in s]
            if len(weirdArgs) > 0: raise Exception(f"Custom datatype `{type(fn)}` has ._cat() method, which expects only `ser` and `kwargs` arguments, but detected these arguments instead: {weirdArgs}. Please fix `{type(fn)}`")
            def guard():
                if kwdiff: raise Exception(f"Custom datatype `{type(fn)}` does not support custom cat() arguments like {kwdiff}")
            if n == 0: guard(); return fn._cat() | ser
            elif n == 1:
                if args[0] == "ser": guard(); return fn._cat(ser)
                if args[0] == "kwargs": return fn._cat(kwargs) | ser
            elif n == 2:
                if args[0] == "ser": return fn._cat(ser, kwargs)
                else: return fn._cat(kwargs, ser)
            else: raise Exception("Unreachable")
        fn = os.path.expanduser(fn) if isinstance(fn, str) else fn
        if text and chunks and k1lib._settings.packages.k1a and isinstance(fn, str) and os.path.exists(fn):
            return k1lib._k1a.k1a.StrIterCat(fn, sB, eB) | ser # accelerated C version
        if chunks: return _catGenText(fn, sB, eB) if text else _catGenBin(fn, sB, eB) | ser
        sB = wrap(fn, sB); eB = wrap(fn, eB)
        if text:
            with openFile(fn, True) as f: f.seek(sB); return f.read(eB-sB).splitlines() | ser
        else:
            with openFile(fn, False) as f: f.seek(sB); return f.read(eB-sB) | ser
class Profile(BaseCli):
    def __init__(self, text): self.data = []; self.text = text
    def __ror__(self, it):
        fmt = k1lib.fmt; chars = 0; beginTime = time.time()
        if self.text:
            a, b, c, d, f = k1lib.ConstantPad.multi(5); every = settings.cat.every.text
            for lines, e in enumerate(it):
                chars += len(e)
                if lines % every == 0: # every 1000 lines, print stuff out
                    elapsed = time.time() - beginTime#; self.data.append([lines, chars, elapsed])
                    print(f"Current line: {fmt.item(lines) | a} ({fmt.item(lines/elapsed) | b} lines/s), current byte/chars: {fmt.size(chars) | c} ({fmt.size(chars/elapsed) | d}/s), elapsed: {fmt.time(elapsed) | f}                                 ", end="\r")
                yield e
        else:
            a, b, c = k1lib.ConstantPad.multi(3); every = settings.cat.every.binary
            for i, e in enumerate(it):
                chars += len(e)
                if i % every == 0: # every 10 100000-byte chunks, print stuff out
                    elapsed = time.time() - beginTime#; self.data.append([chars, elapsed])
                    print(f"Current size/chars: {fmt.size(chars) | a} ({fmt.size(chars/elapsed) | b}/s), elapsed: {fmt.time(elapsed) | c}                                 ", end="\r")
                yield e
def cat(fileName:str=None, text:bool=True, chunks:bool=None, profile:bool=False, sB=0, eB=-1):
    """Reads a file line by line.
Example::

    # display first 10 lines of file
    cat("file.txt") | headOut()
    # piping in also works
    "file.txt" | cat() | headOut()
    
    # read bytes from an image file and dumps it to another file
    cat("img.png", False) | file("img2.png")

If you want to read only specific sections of the file, you can specify the
start (``sB``) and end byte (``eB``) like this::

    "123456\\n89" | file("test/catTest.pth")
    # returns ['3456', '8']
    cat("test/catTest.pth", sB=2, eB=8) | deref()
    
    settings.cat.context.chunkSize=3 # just for demonstration, don't do it normally
    # returns [b'123', b'456', b'\\n8']
    cat("test/catTest.pth", text=False, chunks=True, eB=8) | deref()

.. admonition:: Remote files

    You can also read from urls directly, like this::

        cat("https://k1lib.com/latest/") | deref()

    For remote files like this, there are extra settings at :data:`~k1lib.settings`.cli.RemoteFile.
    This will also read the file chunk by chunk if required. If the website doesn't support
    partial downloads, then all of it will be downloaded and stored into ram, which may not be
    desireable. The available settings are:
    
    - timeout: seconds before killing the existing request
    - retries: try to resend the request for this much before giving up

If you are working with large files and would like to read 1 file from multiple
threads/processes, then you can use this cli in conjunction with :class:`splitSeek`.

If you are dumping multiple pickled objects into a single file, you can read all
of them using :meth:`cat.pickle`.

This cli has lots of settings at :data:`~k1lib.settings`.cli.cat

See also: :meth:`ls`

.. admonition:: Custom datatype

    It is possible to build objects that can interoperate with this cli,
    like this::

        class custom1:
            def __init__(self, config=None): ...
            def _cat(self): return ["abc", "def"]

        custom1() |  cat()           # returns ["abc", "def"]
        custom1() |  cat() | item()  # returns "abc"
        custom1() | (cat() | item()) # returns "abc"

    When called upon, :meth:`cat` will see that the input is not a simple string, which
    will prompt it to look for ``_cat()`` method of the complex object and execute it.
    By default, if the user specifies any non-default arguments like ``text=False``,
    it will errors out because :meth:`cat` does not know how to handle it. Here's how
    to do it right::

        class custom2:
            def __init__(self, config=None): ...
            def _cat(self, kwargs): # default kwargs if user doesn't specify anything else is `{"text": True, "chunks": None, "sB": 0, "eB": -1}`
                if kwargs["text"]: return ["abc", "def"]
                else: return [b"abc", b"def"]
        
        custom2() |  cat()                     # returns ["abc", "def"]
        custom2() |  cat()           | item()  # returns "abc"
        custom2() | (cat()           | item()) # returns "abc"
        custom2() |  cat(text=False)           # returns [b"abc", b"def"]
        custom2() |  cat(text=False) | item()  # returns b"abc"

    Here, you're saying that your function can handle non-standard arguments, so
    :meth:`cat` will give you all the args. You may support only some arguments
    and completely ignore others, it's all up to you. Might still be worth it to
    throw some errors warning users of arguments your custom datatype does not support.
    
    You can also capture future clis like this::

        class custom3:
            def __init__(self, config=None): ...
            def _cat(self, ser):            # "ser" stands for "serial"
                if len(ser.clis) == 1 and isinstance(ser.clis[0], item):
                    return "123"            # fancy optimization
                return ["abc", "def"] | ser # default "slow" base case

        custom3() |  cat()           # returns ["abc", "def"]
        custom3() |  cat() | item()  # returns "abc", because cat() did not capture any clis
        custom3() | (cat() | item()) # returns "123", which might be desireable, up to you

    This feature is pretty advanced actually, in that you can actually do different
    things based on future processing tasks. Let's say that the captured cli looks like
    ``cut(3) | batched(10) | rItem(3)``. This essentially means "give me elements 30
    through 39 from the 4th column". With this information, you can query your custom
    database for exactly those elements only, while fetching nothing else, which would be
    great for performance.
    
    You can also outright lie to the user like in the example, where if :class:`~k1lib.cli.utils.item`
    is detected, it will return a completely different version ("123") while it should return
    "abc" instead. The possibilities are endless. As you can probably feel, it's super hard to
    actually utilize this in the right way without breaking something, as you are completely
    responsible for the captured clis. Let's see another example::

        class custom4:
            def __init__(self, config=None): ...
            def _cat(self, ser):
                return ["abc", "def"]
        
        custom4() |  cat()           # returns ["abc", "def"]
        custom4() |  cat() | item()  # returns "abc"
        custom4() | (cat() | item()) # returns ["abc", "def"]

    Same story as ``custom3``, demonstrating that if you declare that you want to see and
    manipulate ``ser``, you'll be completely responsible for it, and if you don't handle it
    correctly, it will create horrible bugs. You can, of course, have access to ``ser`` and
    ``kwargs`` at the same time::

        class custom5:
            def __init__(self, config=None): ...
            def _cat(self, ser, kwargs): return ["abc", "def"] | ser
    
    If your custom datatype feels like a block device, and you don't want to rewrite all
    the functionalities in :meth:`cat`, you can implement the method ``_catHandle`` that
    yields the custom file handle instead::
    
        class custom6:
            def __init__(self, config=None): ...
            def _catHandle(self): yield io.BytesIO(b"abc\\ndef\\n123")
        class custom7:
            def __init__(self, config=None): ...
            def _catHandle(self): yield io.StringIO("abc\\ndef\\n123")

        custom6() | cat(text=False) # returns b'abc\\ndef\\n123'
        custom7() | cat() | deref() # returns ['abc', 'def', '123']

    Remember that you have to yield instead of returning the file handle. This is so that
    you can use ``with`` statements, and if you return the file handle, the file might be
    closed by the time :meth:`cat` decides to use it.

:param fileName: if None, then return a :class:`~k1lib.cli.init.BaseCli`
    that accepts a file name and outputs Iterator[str]
:param text: if True, read text file, else read binary file
:param chunks: if True then reads the file chunk by chunk, else reads the
    entire file. Defaults to True in text mode and False in binary mode
:param profile: whether to profile the file reading rate or not. Can adjust
    printing frequency using `settings.cli.cat.every`
:param sB: "start byte". Specify this if you want to start reading from this byte
:param eB: "end byte", exclusive. Default -1 means end of file"""
    if chunks is None: chunks = True if text else False # dev note: if default arguments are changed, please also change _cat()'s implementation for custom datatypes
    if profile and not chunks: warnings.warn(f"Can't profile reading rate when you're trying to read everything at once"); profile = False
    f = _cat(text, chunks, sB, eB)
    if profile: f = f | Profile(text)
    return f if fileName is None else fileName | f
def _catPickle(fileName=None, pickleModule=dill):
    """Reads a file as a series of pickled objects.
Example::

    "ab" | aS(dill.dumps) | file("test/catTest.pth")
    "cd" | aS(dill.dumps) >> file("test/catTest.pth") # append to the file
    # returns ["ab", "cd"]
    cat.pickle("test/catTest.pth") | deref()
    # also returns ["ab", "cd"], same style as :class:`cat`
    "test/catTest.pth" | cat.pickle() | deref()

:param fileName: name of the pickled file
:param pickleModule: pickle module to use. Python's default is "pickle", but
    I usually use :mod:`dill` because it's more robust"""
    def gen(fn):
        with open(os.path.expanduser(fn), "rb") as f:
            try:
                while True: yield dill.load(f)
            except: pass
    return cli.aS(gen) if fileName is None else fileName | cli.aS(gen)
cat.pickle = _catPickle

In [7]:
k1lib._settings.packages.k1a = True
assert len("inp.py" | cat() | cli.head() | cli.toList()) == 10
assert len(cat("inp.py") | cli.head() | cli.toList()) == 10
k1lib._settings.packages.k1a = False
assert len("inp.py" | cat() | cli.head() | cli.toList()) == 10
assert len(cat("inp.py") | cli.head() | cli.toList()) == 10
assert cat("~/ssd/data/pubchem/Compound/CID-SMILES", profile=True) | cli.head(int(1e6)) | cli.shape(0) == 1000000
assert cat("~/ssd/data/pubchem/Compound/CID-SMILES", False, True, True) | cli.head(10000) | cli.shape(0) == 10000
"123456\n89" | cli.file("test/catTest.pth")
assert fileLength("test/catTest.pth") == 10
assert cat("test/catTest.pth", eB=9) | cli.deref() == ['123456', '89']
assert cat("test/catTest.pth", eB=8) | cli.deref() == ['123456', '8']
assert cat("test/catTest.pth", eB=7) | cli.deref() == ['123456', '']
assert cat("test/catTest.pth", eB=6) | cli.deref() == ['123456']
assert cat("test/catTest.pth", sB=2, eB=8) | cli.deref() == ['3456', '8']
assert cat("test/catTest.pth", False) == b'123456\n89\n'
assert cat("test/catTest.pth", False, eB=9) == b'123456\n89'
assert cat("test/catTest.pth", False, eB=8) == b'123456\n8'
assert cat("test/catTest.pth", False, True, eB=9) | cli.deref() == [b'123456\n89']
assert cat("test/catTest.pth", False, True, eB=8) | cli.deref() == [b'123456\n8']
assert cat("test/catTest.pth", False, True, eB=6) | cli.deref() == [b'123456']
assert cat("test/catTest.pth", False, True, sB=7, eB=6) | cli.deref() == []
with settings.cat.context(chunkSize=3):
    assert (cat("test/catTest.pth", False, True, eB=9) | cli.deref() == [b'123', b'456', b'\n89'])
    assert (cat("test/catTest.pth", False, True, eB=8) | cli.deref() == [b'123', b'456', b'\n8'])
    assert (cat("test/catTest.pth", False, True, eB=6) | cli.deref() == [b'123', b'456'])
    assert (cat("test/catTest.pth", False, True, sB=7, eB=6) | cli.deref() == [])

In [8]:
"ab" | cli.aS(dill.dumps) | cli.file("test/catTest.pth")
"cd" | cli.aS(dill.dumps) >> cli.file("test/catTest.pth")
assert cat.pickle("test/catTest.pth") | cli.deref() == ["ab", "cd"]
assert "test/catTest.pth" | cat.pickle() | cli.deref() == ["ab", "cd"]
url = "https://vim.kelvinho.org/.vimrc"
assert cat(url, sB=2, eB=20) | cli.item() == "ntax on"
assert cat("https://k1lib.com/latest/", profile=True) | cli.deref() | cli.shape(0) > 0
assert "test/nucleo-free-sample.zip" | cli.ls() | ~cli.grep(" 0.0 B") | cli.shape(0) == 2344
assert "test/nucleo-free-sample.zip" | cli.ls() | ~cli.grep(" 0.0 B") | cli.item() | cat(text=False) | cli.shape(0) == 312
try: "test/nucleo-free-sample.zip" | cli.ls() | ~cli.grep(" 0.0 B") | cli.item() | cat() | cli.deref(); assert False
except UnicodeDecodeError: pass

In [9]:
class custom1:
    def __init__(self, config=None): ...
    def _cat(self): return ["abc", "def"]
assert custom1() |  cat()           == ["abc", "def"]
assert custom1() |  cat() | cli.item()  == "abc"
assert custom1() | (cat() | cli.item()) == "abc"
class custom2:
    def __init__(self, config=None): ...
    def _cat(self, kwargs): # default kwargs if user doesn't specify anything else is `{"text": True, "chunks": None, "sB": 0, "eB": -1}`
        if kwargs["text"]: return ["abc", "def"]
        else: return [b"abc", b"def"]
assert custom2() |  cat()                         == ["abc", "def"]
assert custom2() |  cat()           | cli.item()  == "abc"
assert custom2() | (cat()           | cli.item()) == "abc"
assert custom2() |  cat(text=False)               == [b"abc", b"def"]
assert custom2() |  cat(text=False) | cli.item()  == b"abc"
class custom3:
    def __init__(self, config=None): ...
    def _cat(self, ser):
        if len(ser.clis) == 1 and isinstance(ser.clis[0], cli.item): return "123"
        else: return ["abc", "def"] | ser
assert custom3() |  cat()               == ["abc", "def"]
assert custom3() |  cat() | cli.item()  == "abc"
assert custom3() | (cat() | cli.item()) == "123"
class custom4:
    def __init__(self, config=None): ...
    def _cat(self, ser): return ["abc", "def"]
assert custom4() |  cat()               == ["abc", "def"]
assert custom4() |  cat() | cli.item()  == "abc"
assert custom4() | (cat() | cli.item()) == ["abc", "def"]
class custom5:
    def __init__(self, config=None): ...
    def _cat(self, ser, kwargs): return ["abc", "def"] | ser
class custom6:
    def __init__(self, config=None): ...
    def _catHandle(self): yield io.BytesIO(b"abc\ndef\n123")
assert custom6() | cat(text=False) == b'abc\ndef\n123'
class custom7:
    def __init__(self, config=None): ...
    def _catHandle(self): yield io.StringIO("abc\ndef\n123")
assert custom7() | cat() | cli.deref() == ['abc', 'def', '123']

In [10]:
#export
class splitSeek(BaseCli):
    def __init__(self, n=None, c=b'\n', ws=None):
        """Splits a file up into n fragments aligned to the closest character and return the seek points.
Example::

    # preparing the large file
    range(120) | apply(lambda x: f"{x:3}_56789") | file("test/largeFile.txt")
    # returns [0, 30, 70, 110, 150, 190, 230, 270, 300, 340]
    "test/largeFile.txt" | splitSeek(31) | head()
    # returns 32
    "test/largeFile.txt" | splitSeek(31) | shape(0)
    # returns 32, demonstrating you can also pipe file objects in, if that's what you want
    open("test/largeFile.txt") | splitSeek(31) | shape(0)
    # returns [0, 0, 10, 10, 20, 30, 30, 40, 40, 50], notice some segments have zero length
    "test/largeFile.txt" | splitSeek(200) | head()
    # returns [0, 400, 1200], demonstrating that you can split a file up unevenly by weights
    "test/largeFile.txt" | splitSeek(ws=[1, 2]) | deref()

So, the generated file has 120 lines in total. Each line is 10 bytes (9 for
the string, and 1 for the new line character). Splitting the file into 31
fragments will result in 32 seek points (:math:`p_i\quad i\in[1, n+1]`). You
can then use these seek points to read the file in multiple threads/processes
using :meth:`cat`, like this::

    # returns [['  0_56789', '  1_56789', '  2_56789'], ['  3_56789', '  4_56789', '  5_56789', '  6_56789']]
    "test/largeFile.txt" | splitSeek(31) | window(2) | ~apply(lambda sB, eB: cat("test/largeFile.txt", sB=sB, eB=eB-1)) | head(2) | deref()

Because :math:`120/31\\approx4`, most of cat's reads contain 4 lines, but some
has 3 lines. Also notice that the lines smoothly transitions between cat's
reads (``2_56789`` to ``3_56789``), so that's pretty nice. Just like with :meth:`cat`,
this also works with urls::

    "https://example.com" | splitSeek(10)

.. warning::

    You have to really test whether reading the same file from multiple processes is
    going to be really faster or not. If your data is stored in a HDD (aka hard drive,
    with spinning disks), then it will actually slow you down (10x-100x), because the
    disk will have to context switch all the time, and each switch has a 10ms cost.
    
    You also have to take into account collecting together the results of all processes,
    which can bottleneck the cpu. Read more about concurrency pitfalls at :class:`~k1lib.cli.modifier.applyMp`.

In some scenarios where you want to adjust the seek points even more, like when
you want to parse FASTA genome files, which has blocks of 2/4 lines each like this:

.. code-block:: text

    @FP200005993L1C001R00100000061/2
    TTTTAAACTTGCATTCTTTGGAGATTTGCTGAGTGTTGCTAGAGCTGGGAAACTTTTTTAATGAGATACGTGCATATTTTTCAAATTTACAGATCTTTTTTCACAAAAATAGAAAGTCATAAATGTGAAATGGAAACCTAAACAAGGCAA
    +
    GFEEEDEFGFFFFEFFFFFIFCEEEFFFGFFDFEGEDGFFFGDGFFGDGCE@GGGEEFDFGFGCFDFGGHCHFFFGFFFFFGEFDFFGHGFGEEHGFGEGFGFHFFEGFFFE;GEGEFGGHFFEI=GDAEDIFDDFGHFGEFGFEGGGGF
    @FP200005993L1C001R00100000167/2
    CTGGAATTTGGTATCTTATTGCCAAAGAATCTGTTTTGTGAAACTTGGGATCTCTATTTTAATGTTAATTCTGGTCAGTTGTGCCTAAACTCCATAAAGCAGGGACTATACTGAGGCGTATTCAATCTTCCTTCTTACCAAGGCCAGGAA
    +
    EEFECEDEFFCGFFFFFEEEGEGFEDECCEFEFDFEEFDFEDDFEFEEFDDFFEEFFEEFEFFHEEFEEFEFFDEFFFECF>FFFEFEDFCFFFEGFEDEEGDDFEEFEFGEEBD@EG>EEFFECEEGFEEEFFEDGEEEDE5EBDG:CC

Here, each 4 lines are (title, read, blank, quality). Because by default, this will
only split neatly along new lines, you will have to write extra functions to detect
if a particular seek point is desirable, and if not, either jump forward or backward
using :meth:`splitSeek.forward` and :meth:`splitSeek.backward`. To help with this,
:class:`refineSeek` has some useful methods that you might want to check out.

:param n: how many splits do you want?
:param c: block-boundary character, usually just the new line character
:param ws: weights. If given, the splits length ratios will roughly correspond to this"""
        self.n = n; self.c = c; self.ws = ws; self.fn = None; self.res = None # file name, result
        if ws is None and n is None: raise Exception("Specify at least n or ws for splitSeek to work")
    @staticmethod
    def forward(f, i:int, c=b'\n') -> int:
        """Returns char location after the search char, going forward.
Example::

    f = io.BytesIO(b"123\\n456\\n789\\nabc")
    f | splitSeek(2) # returns [0, 4, 15]
    splitSeek.forward(f, 2) # returns 4
    splitSeek.forward(f, 3) # returns 4
    splitSeek.forward(f, 4) # returns 8

:param f: file handle
:param i: current seek point
:param c: block-boundary character"""
        def inner(f):
            f.seek(i)
            while True:
                b = f.tell(); s = f.read(1000); di = s.find(c)
                if di > -1: return b + di + 1
                if s == "": return -1
        if isinstance(f, str):
            with openFile(os.path.expanduser(f), False) as _f: return inner(_f)
        else: return inner(f)
    @staticmethod
    def backward(f, i:int, c=b'\n') -> int:
        """Returns char location after the search char, going backward.
Example::

    f = io.BytesIO(b"123\\n456\\n789\\nabc")
    f | splitSeek(2) # returns [0, 4, 15]
    splitSeek.backward(f, 5) # returns 4
    splitSeek.backward(f, 4) # returns 4
    splitSeek.backward(f, 3) # returns 0

:param f: file handle
:param i: current seek point
:param c: block-boundary character"""
        def inner(f):
            mul = 1
            while True:
                begin = max(i-1000*mul, 0); end = max(i-1000*(mul-1), 0); mul += 1 # search range
                f.seek(begin); b = f.tell(); s = f.read(end-begin); di = s.rfind(c)
                if di > -1: return b + di + 1
                if b == 0: return 0
        if isinstance(f, str):
            with openFile(os.path.expanduser(f), False) as _f: return inner(_f)
        else: return inner(f)
    def __ror__(self, fn): # why return self instead of the seek positions directly? Because we want to pass along dependencies like file name to downstream processes like refineSeek
        if isinstance(fn, str): fn = os.path.expanduser(fn)
        n = self.n; c = self.c; ws = self.ws; self.fn = fn
        def func(f):
            f.seek(0, os.SEEK_END); end = f.tell()
            if ws is None: begins = range(n) | cli.apply(lambda x: int(x*end/n))
            else: begins = range(end) | cli.splitW(*ws) | cli.apply(lambda x: x.start)
            return [*begins | cli.apply(lambda x: splitSeek.backward(f, x, c)), end]
        if isinstance(fn, str):
            with openFile(os.path.expanduser(fn), False, True) as f: self.res = func(f); return self
        else: self.res = func(fn); return self
    def __or__(self, aft):
        if self.res is None: return super().__or__(aft)
        return aft.__ror__(self)
    def __getitem__(self, idx): return self.res[idx]
    def __len__(self): return len(self.res)
    def __iter__(self): return iter(self.res)
    def __repr__(self): return self.res.__repr__()

In [11]:
range(120) | cli.apply(lambda x: f"{x:3}_56789") | cli.file("test/largeFile.txt")
assert "test/largeFile.txt" | splitSeek(31) | cli.head() == [0, 30, 70, 110, 150, 190, 230, 270, 300, 340]
assert "test/largeFile.txt" | splitSeek(31) | cli.shape(0) == 32
assert "test/largeFile.txt" | splitSeek(200) | cli.head() == [0, 0, 10, 10, 20, 30, 30, 40, 40, 50]
assert "test/largeFile.txt" | splitSeek(31) | cli.window(2) | ~cli.apply(lambda sB, eB: cat("test/largeFile.txt", sB=sB, eB=eB-1)) | cli.head(2) | cli.deref() == [['  0_56789', '  1_56789', '  2_56789'], ['  3_56789', '  4_56789', '  5_56789', '  6_56789']]
assert "test/largeFile.txt" | splitSeek(ws=[1, 2]) | cli.deref() == [0, 400, 1200]
f = io.BytesIO(b"123\n456\n789\nabc"); assert f | splitSeek(2) | cli.deref() == [0, 4, 15]
assert splitSeek.forward(f,  2) == 4; assert splitSeek.forward(f,  3) == 4; assert splitSeek.forward(f,  4) == 8
assert splitSeek.backward(f, 5) == 4; assert splitSeek.backward(f, 4) == 4; assert splitSeek.backward(f, 3) == 0
assert splitSeek.forward("test/largeFile.txt", 30) == 40
assert splitSeek.backward("test/largeFile.txt", 29) == 20
url = "https://vim.kelvinho.org/.vimrc"
b = url | splitSeek(10) | cli.window(2) | cli.deref() | ~cli.apply(lambda sB, eB: cat(url, sB=sB, eB=eB-1)) | cli.joinStreams() | cli.deref()
a = cat(url) | cli.deref(); assert a == b

In [12]:
#export
class refineSeek(BaseCli):
    def __init__(self, f=None, window=1):
        """Refines seek positions.
Example::

    # returns list of integers for seek positions
    "abc.txt" | splitSeek(30)
    # returns refined seek positions, such that the line starting at the seek positions starts with "@"
    "abc.txt" | splitSeek(30) | refineSeek(lambda x: x.startswith(b"@"))
    # same thing as above
    "abc.txt" | splitSeek(30) | refineSeek(lambda x: x[0] == b"@"[0])
    # returns refined seek positions, such that 0th line starts with "@" and 2nd line starts with "+". This demonstrates `window` parameter
    "abc.txt" | splitSeek(30) | refineSeek(lambda x: x[0][0] == b"@"[0] and x[2][0] == b"+"[0], 3)
    # same thing as above, demonstrating some builtin refine seek functions
    "abc.txt" | splitSeek(30) | refineSeek.fastq()

:param f: function that returns True if the current line/lines is a valid block boundary
:param window: by default (1), will fetch 1 line and check boundary using ``f(line)``.
    If a value greater than 1 is passed (for example, 3), will fetch 3 lines and check
    boundary using ``f([line1, line2, line3])``
"""
        if f is None: f = lambda x: True
        self.f = cli.fastF(f); self.window = window; self.fn = None
    def __ror__(self, seeks):
        f = self.f; window = self.window
        def read(fio, sB:int):
            fio.seek(sB)
            if window == 1: return fio.readline()
            return list(cli.repeatF(lambda: fio.readline(), window))
        if len(seeks) <= 2: return seeks
        fn = self.fn or seeks.fn; newSeeks = [seeks[0]]
        def process(fio):
            with openFile(fn, False) as fio:
                for seek in seeks[1:-1]:
                    line = read(fio, seek)
                    while not f(line): seek = splitSeek.forward(fio, seek); line = read(fio, seek)
                    newSeeks.append(seek)
            newSeeks.append(seeks[-1]); return newSeeks
        if isinstance(fn, str):
            with openFile(fn, False) as fio: return process(fio)
        else: return process(fn)
    def injectFn(self, fn):
        """Injects file name dependency, if this is not used right after :class:`splitSeek`"""
        self.fn = fn; return self
    @classmethod
    def fastq(cls):
        """Refine fastq file's seek points"""
        return cls(lambda x: x[0][0] == b"@"[0] and x[2][0] == b"+"[0], 3)

In [13]:
def readline(fn:str, sB:int):
    with open(os.path.expanduser(fn), "rb") as f: f.seek(sB); return f.readline()
fn = "~/hdd/genome/NG156Q3CQM_2.fq"
assert fn | splitSeek(31) | refineSeek.fastq() | cli.apply(lambda sB: readline(fn, sB)) | cli.head(-1) | cli.apply(lambda x: x.startswith(b"@")) | cli.aS(all)

In [14]:
#export
def wget(url:str, fileName:str=None, mkdir=True):
    """Downloads a file. Also returns the file name, in case you want to pipe it
to something else.

:param url: The url of the file
:param fileName: if None, then tries to infer it from the url
:param mkdir: whether to make the directory leading up to the file or not"""
    if fileName is None: fileName = url.split("/")[-1]
    fileName = os.path.expanduser(fileName); dirname = os.path.dirname(fileName)
    if mkdir: os.makedirs(dirname, exist_ok=True)
    try: urllib.request.urlretrieve(url, fileName); return fileName
    except: return None

In [15]:
#export
def ls(folder:str=None):
    """List every file and folder inside the specified folder.
Example::

    # returns List[str]
    ls("/home")
    # same as above
    "/home" | ls()
    # only outputs files, not folders
    ls("/home") | filt(os.path.isfile)

This can handle things that are not plain folders. For example,
it can handle zip file whereby it will list out all the files contained
within a particular .zip::

    ls("abc.zip")

Then, you can use :meth:`cat` as usual, like this::

    ls("abc.zip") | item() | cat()

Pretty nice!

.. admonition:: Custom datatype

    It is possible to build objects that can interoperate with this cli,
    like this::
    
        class sql:
            def __init__(self): ...
            def _ls(self): return ["something", "here"]

    ls() will identify that what's inputted is not a string, and will try
    to execute the object's "_ls()" method, so you can just simply implement
    it in your classes
"""
    if folder is None: return _ls()
    else: return folder | _ls()
class _ls(BaseCli):
    def _typehint(self, ignored=None): return tList(str)
    def __ror__(self, path:str):
        if isinstance(path, str):
            path = os.path.expanduser(path.rstrip(os.sep))
            if os.path.exists(path):
                if os.path.isfile(path):
                    if cat(path, False, eB=2) == b"PK": # list subfiles in a zip file
                        return [ZipWrapper(e, path) for e in zipfile.ZipFile(path).infolist()]
                    else: raise Exception(f"{path} is a file, not a folder, so can't list child directories")
                else: return [f"{path}{os.sep}{e}" for e in os.listdir(path)]
            else: return []
        else: return path._ls()

In [16]:
assert len(ls("/home")) == len("/home" | ls())
assert len(ls("/home")) > 0
assert len("/home/kelvin" | ls() | cli.filt(os.path.isfile) | cli.deref()) > 0
assert len("~" | ls() | cli.filt(os.path.isfile) | cli.deref()) > 0

In [17]:
#export
k1lib.settings.cli.add("quiet", False, "whether to mute extra outputs from clis or not")
newline = b'\n'[0]
class lazySt:
    def __init__(self, st, text:bool):
        """Converts byte stream into lazy text/byte stream, with nice __repr__."""
        self.st = st; self.text = text;
    def __iter__(self):
        if self.text:
            while True:
                line = self.st.readline()
                if len(line) == 0: break
                yield line.decode().rstrip("\n")
        else:
            while True:
                line = self.st.readline()
                if len(line) == 0: break
                yield line
    def __repr__(self): self | cli.stdout(); return ""
def executeCmd(cmd:str, inp:bytes, text):
    """Runs a command, and returns stdout and stderr streams"""
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=k1lib.settings.wd)
    if inp is not None:
        if isinstance(inp, (str, bytes)): p.stdin.write(inp if isinstance(inp, bytes) else inp.encode())
        else:
            for e in inp:
                if not isinstance(e, (str, bytes)): e = str(e)
                if not isinstance(e, bytes): e = e.encode()
                p.stdin.write(e); p.stdin.write(b"\n")
    p.stdin.close(); return p, lazySt(p.stdout, text), lazySt(p.stderr, text)
def printStderr(err):
    if not k1lib.settings.cli.quiet:
        e, it = err | cli.peek()
        if it != []: it | cli.insert("\nError encountered:\n") | cli.apply(k1lib.fmt.txt.red) | cli.stdout()
def requireCli(cliTool:str):
    """Searches for a particular cli tool (eg. "ls"), throws ImportError if not
found, else do nothing"""
    a = cmd(cliTool); None | a;
    if len(a.err) > 0: raise ImportError(f"""Can't find cli tool {cliTool}. Please install it first.""")

In [18]:
#export
class cmd(BaseCli):
    def __init__(self, cmd:str, mode:int=1, text=True, block=False): # 0: return (stdout, stderr). 1: return stdout, 2: return stderr
        """Runs a command, and returns the output line by line. Can pipe in some
inputs. If no inputs then have to pipe in :data:`None`. Example::

    # return detailed list of files
    None | cmd("ls -la")
    # return list of files that ends with "ipynb"
    None | cmd("ls -la") | cmd('grep ipynb$')

It might be tiresome to pipe in :data:`None` all the time. So, you can use ">"
operator to yield values right away::

    # prints out first 10 lines of list of files
    cmd("ls -la") > headOut()

If you're using Jupyter notebook/lab, then if you were to display a :class:`cmd`
object, it will print out the outputs. So, a single command ``cmd("mkdir")``
displayed at the end of a cell is enough to trigger creating the directory.

Reminder that ">" operator in here sort of has a different meaning to that of
:class:`~k1lib.cli.init.BaseCli`. So you kinda have to becareful about this::

    # returns a serial cli, cmd not executed
    cmd("ls -la") | deref()
    # executes cmd with no input stream and pipes output to deref
    cmd("ls -la") > deref()
    # returns a serial cli
    cmd("ls -la") > grep("txt") > headOut()
    # executes pipeline
    cmd("ls -la") > grep("txt") | headOut()

General advice is, right ater a :class:`cmd`, use ">", and use "|" everywhere else.

Let's see a few more exotic examples. File ``a.sh``:

.. code-block:: bash

    #!/bin/bash

    echo 1; sleep 0.5
    echo This message goes to stderr >&2
    echo 2; sleep 0.5
    echo $(</dev/stdin)
    sleep 0.5; echo 3

Examples::

    # returns [b'1\\n', b'2\\n', b'45\\n', b'3\\n'] and prints out the error message
    "45" | cmd("./a.sh", text=False) | deref()
    # returns [b'This message goes to stderr\\n']
    "45" | cmd("./a.sh", mode=2, text=False) | deref()
    # returns [[b'1\\n', b'2\\n', b'45\\n', b'3\\n'], [b'This message goes to stderr\\n']]
    "45" | cmd("./a.sh", mode=0, text=False) | deref()

Performance-wise, stdout and stderr will yield values right away as soon
as the process outputs it, so you get real time feedback. However, this will
convert the entire input into a :class:`bytes` object, and not feed it bit by
bit lazily, so if you have a humongous input, it might slow you down a little.

Also, because stdout and stderr yield values right away, it means that if you
want the operation to be blocking until finished, you have to consume the output::

    None | cmd("mkdir abc")
    # might fail, because this might get executed before the previous line
    None | cmd("echo a>abc/rg.txt")
    
    None | cmd("mkdir abc") | ignore()
    # will succeed, because this will be guaranteed to execute after the previous line
    None | cmd("echo a>abc/rg.txt")

Settings:
- cli.quiet: if True, won't display errors in mode 1

:param mode: if 0, returns ``(stdout, stderr)``. If 1, returns ``stdout`` and prints
    ``stderr`` if there are any errors. If 2, returns ``stderr``
:param text: whether to decode the outputs into :class:`str` or return raw :class:`bytes`
:param block: whether to wait for the task to finish before returning to Python or not"""
        super().__init__(); self.cmd = cmd; self.mode = mode
        self.text = text; self.block = block; self.ro = k1lib.RunOnce()
    def _typehint(self, ignored=None):
        t = tIter(str) if self.text else tIter(bytes)
        if self.mode == 0: return tCollection(t, t)
        return t
    def __ror__(self, it:Union[None, str, bytes, Iterator[Any]]) -> Iterator[Union[str, bytes]]:
        """Pipes in lines of input, or if there's nothing to
pass, then pass None"""
        if not self.ro.done(): self.p, self.out, self.err = executeCmd(self.cmd, it, self.text); mode = self.mode
        if self.block:
            self.out = self.out | cli.deref()
            self.err = self.err | cli.deref()
        if mode == 0: return (self.out, self.err)
        elif mode == 1:
            threading.Thread(target=lambda: printStderr(self.err)).start()
            return self.out
        elif mode == 2: return self.err
    def __gt__(self, it): return None | self | it
    def __repr__(self):
        return (None | self).__repr__()

In [19]:
assert None | tCheck() | cmd("ls -la") | cli.shape(0) | (cli.op() > 0)
assert None | tCheck() | cmd("ls -la", block=True) | cli.shape(0) | (cli.op() > 0)
assert None | tCheck() | cmd("ls -la") | cmd('grep ipynb$') | cli.shape(0) | (cli.op() > 0)
with k1lib.captureStdout() as out: cmd("ls -la") > cli.headOut()
assert len(out()) > 3
assert type(cmd("ls -la") | cli.deref()).__name__ == "serial"
assert len(cmd("ls -la") > cli.deref()) > 0
assert None | cmd("ls -la") | cmd("grep ipynb") | cli.deref() | cli.shape(0) > 0
with k1lib.captureStdout() as out: assert "45" | tCheck() | cmd("test/a.sh", text=False) | cli.deref() == [b'1\n', b'2\n', b'45\n', b'3\n']
assert len(out()) > 2
assert "45" | tCheck() | cmd("test/a.sh", mode=2, text=False) | cli.deref() == [b'This message goes to stderr\n']
assert "45" | tCheck() | cmd("test/a.sh", mode=0, text=False) | cli.deref() == ([b'1\n', b'2\n', b'45\n', b'3\n'], [b'This message goes to stderr\n'])

In [20]:
#export
class walk(BaseCli):
    def __init__(self, **kwargs):
        """Recursively get all files inside a dictionary.
Example::

    # prints out first 10 files
    "." | walk() | headOut()"""
        self.kwargs = kwargs
    def __ror__(self, path):
        return os.walk(path, **self.kwargs) | ~cli.apply(lambda x, y, z: z | cli.apply(lambda e: x + os.sep + e)) | cli.joinStreams()

In [21]:
assert "." | walk() | cli.shape(0) > 0

In [22]:
#export
def urlPath(base:str, host:bool=True):
    """Translates from a url to a file path.
Example::

    base = "~/ssd/some/other/path"
    url = "http://example.com/some/path/to/file.txt"
    url | urlPath(base) # returns "~/ssd/some/other/path/example_com/some/path/to/file.txt"
    url | urlPath(base, False) # returns "~/ssd/some/other/path/some/path/to/file.txt"

:param base: base directory you want the files to be in"""
    base = base.rstrip("/\\")
    def inner(url):
        p = urllib.parse.urlparse(url)
        a = (p.netloc.replace(".", "_") + os.sep) if host else ""
        return f"{base}{os.sep}{a}{p.path.strip('/')}".replace("//", "/")
    return cli.aS(inner)

In [23]:
base = "~/ssd/some/other/path"; url = "http://example.com/some/path/to/file.txt"
assert url | urlPath(base, False) == "~/ssd/some/other/path/some/path/to/file.txt"
assert url | urlPath(base) == "~/ssd/some/other/path/example_com/some/path/to/file.txt"

In [24]:
#export
import bz2, gzip, zlib
class kzip(BaseCli):
    def __init__(self, fmt="gz"):
        """Incrementally compresses a stream of data using gzip or bzip2.
Example::

    data = range(100) | apply(lambda x: str(x).encode()) | deref() # list of bytes
    data | kzip() | deref() # returns list of bytes
    range(100) | apply(str) | kzip() | deref() # returns list of bytes, demonstrating piping iterator of string works

Quick reminder that if you pass in bytes iterator then it will be compressed as-is.
But if you pass in string iterator then it will append a new line character to each
string and then compress it. This is more intuitive, and it makes the style consistent
with :class:`~k1lib.cli.output.file`.

Passing in single string or byte is ok too::

    "0123456789" | kzip()  # returns gz bytes
    b"0123456789" | kzip() # returns gz bytes, exact same pattern as above

Why "kzip" and not just "zip"? Because "zip" is a builtin python keyword.

See also: :class:`kunzip`

:param fmt: desired compressed format. Currently only supports "gz" or "bz2"
"""
        fmt = fmt.strip(".")
        if fmt == "gzip" or fmt == "gz":
            self.o = zlib.compressobj(wbits=31)
        elif fmt == "bzip2" or fmt == "bz2":
            self.o = bz2.BZ2Compressor()
        else: raise Exception(f"File type {fmt} not supported. Specify either 'gz' or 'bz2'")
    def __ror__(self, it):
        o = self.o
        def gen():
            for e in it:
                if isinstance(e, str): e = f"{e}\n".encode()
                res = o.compress(e)
                if res: yield res
            try:
                res = o.flush()
                if res: yield res
            except: pass
        if isinstance(it, str): it = it.encode()
        if isinstance(it, bytes): return [o.compress(it), o.flush()] | cli.filt("x") | cli.aS(b"".join)
        else: return gen()

In [25]:
#export
class decompressobj: # .gz or .bz2 file
    """
Why not just use zlib.decompressobj() directly? I encountered a strange bug when trying
to decompress CommonCrawl's gzip files, posted on Stack Overflow and got help from none
other than Mark Adler, creator of gzip and zlib. That guy's super active on Stack Overflow.
Anyway, this code here is a modified version of his code. Here's the discussion:
https://stackoverflow.com/questions/76452480/pythons-zlib-doesnt-work-on-commoncrawl-file
"""
    def __init__(self, gz=True): self.gz = gz
    def __ror__(self, it):
        gz = self.gz; data = left = b''; o = zlib.decompressobj(zlib.MAX_WBITS|32) if gz else bz2.BZ2Decompressor()
        for got in it:
            yield o.decompress(left + got); left = b''
            if o.eof: left = o.unused_data; o = zlib.decompressobj(zlib.MAX_WBITS|32) if gz else bz2.BZ2Decompressor()
            if len(got) == 0 and len(left) == 0: break
stream_unzip = k1lib.dep("stream_unzip")
class decompressobjZip: # .zip files
    def __ror__(self, it):
        for a, b, c in stream_unzip.stream_unzip(it): yield from c; return
class kunzip(BaseCli):
    def __init__(self, text=False):
        """Incrementally decompress a stream of data using gzip or bzip2.
Example::

    # returns an iterator of bytes. cat() command can shorten to cat("someFile.gz", False, True)
    cat("someFile.gz", text=False, chunks=True) | unzip()
    # incrementally fetches remote file, then incrementally unzips it
    cat("https://example.com/someFile.gz", False, True) | unzip()

    data = range(100) | apply(lambda x: str(x).encode()) | deref() # list of bytes
    data | kzip() | unzip() | deref() # returns original data in list of bytes. May split the bytes at different positions though

How does it know which algorithm to pick to decompress? It looks at the
first few bytes of the file for its magic number. Also, if you're expecting
a text file after decompression, you can do this to get the lines directly::

    # returns iterator of strings
    cat("https://example.com/someFile.gz", False, True) | unzip(True)

One more thing. If you're planning to use this to download whole files, you might
want to tune the chunk size in :data:`~k1lib.settings`.cli.cat.chunkSize as
it might speed things up considerably to raise it. Or may be you should just
use wget instead =))

More examples of the different styles to use this cli::

    ["abc"] | kzip() | unzip(True) | deref() # returns ["abc"]
     "abc"  | kzip() | unzip(True)           # returns  "abc"
     "abc"  | kzip() | unzip()               # returns b"abc"

See also: :class:`kzip`.

.. admonition:: Reading files directly

    The original purpose of this cli is to incrementally decompress a byte stream.
    But a lot of time, that byte stream is coming from a file, and typing out
    ``cat(fn, False, True)`` to generate a byte stream to feed into this cli is
    tedious, so instead, you can pipe in the file name and this will just unzips
    it and return the string/byte stream to you::
    
        "abc.gz"  | unzip() # returns string iterator
        "abc.bz2" | unzip() # same
        "abc.zip" | unzip() # returns string iterator from the first subfile only. All subsequent subfiles are ignored

.. admonition:: .zip files

    This can also work with subfiles within .zip files, like this::

        "abc.zip" | unzip()                   # unzips the first subfile
        "abc.zip" | ls()                      # lists out all subfiles
        "abc.zip" | ls() | rItem(2) | unzip() # unzips the 3rd subfile

:param text: whether to yield string lines or bytes"""
        self.text = text
    def __ror__(self, it):
        def gen(it): # generates a stream of bytes
            it = iter(it); e = next(it)
            if e[:2] == b"\x1f\x8b": o = decompressobj(True) # .gz
            elif e[:3] == b"BZh": o = decompressobj(False)    # .bz2
            elif e[:2] == b"PK": o = decompressobjZip()      # .zip
            else: raise Exception("Can't infer the file type (whether gz or bz2) of this file")
            yield from [[e], it] | cli.joinStreams() | o
        single = False
        if isinstance(it, str): return cat(it, False, True) | kunzip(self.text) # special case for reading file names directly
        if isinstance(it, ZipWrapper): return it | cat(text=self.text, chunks=True) # special case for .zip subfile
        if isinstance(it, bytes): single = True; it = [it]
        if self.text:
            def gen2(it): # generates a stream of strings
                last = b"" # last split is probably not the right line boundary, so gotta do this
                for e in gen(it):
                    e = last + e; splits = e.split(b"\n")
                    for line in splits[:-1]: yield line.decode()
                    last = splits[-1]
                yield last.decode()
            res = gen2(it)
        else: res = gen(it)
        if not single: return res
        else: return "\n".join(res) if self.text else b"".join(res)
unzip = kunzip

In [36]:
data = range(100) | cli.apply(str) | cli.deref() # list of bytes
assert data | kzip() | kunzip() | cli.deref() | cli.aS(b"".join) == (data | cli.apply("x+'\\n'") | cli.join("")).encode() # returns original data
assert data | kzip("bz2") | kunzip() | cli.deref() | cli.aS(b"".join) == (data | cli.apply("x+'\\n'") | cli.join("")).encode()
assert data | kzip() | cli.aS(b"".join) == data | cli.apply("x+'\\n'") | cli.aS("".join) | kzip() | cli.deref()
assert b"0123456789" | kzip() == "0123456789" | kzip()
assert ["abc"] | kzip() | kunzip(True) | cli.deref() == ["abc", ""]
assert "abc" | kzip() | kunzip(True) == "abc"
assert "abc" | kzip() | kunzip() == b"abc"
assert type("test/GSE13277_family.soft.gz" | unzip(True) | cli.item()) == str
assert type("test/GSE13277_family.soft.gz" | unzip() | cli.item()) == bytes
assert type("test/ch04.zip" | unzip() | cli.item()) == bytes
assert type("test/ch04.zip" | ls() | cli.item() | cat(text=False, chunks=True) | cli.item()) == bytes

In [37]:
!../../export.py cli/inp --dist=True

2023-07-15 13:03:43,436	INFO worker.py:1364 -- Connecting to existing Ray cluster at address: 192.168.1.133:6379...
2023-07-15 13:03:43,440	INFO worker.py:1544 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
7131    1   40%   
10664   0   60%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.2
Uninstalling k1lib-1.4.2:
  Successfully uninstalled k1lib-1.4.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
wr